<a href="https://colab.research.google.com/github/andigu/crbm/blob/main/CRBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit emcee numdifftools pyscf > /dev/null
import emcee
import tensorflow as tf 
import tensorflow_probability as tfp
import numpy as np 
from sklearn.neural_network import BernoulliRBM
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import imageio
from datetime import datetime
import os
from qiskit.quantum_info import Statevector, Pauli, Operator
from qiskit.visualization import plot_bloch_multivector, plot_bloch_vector, plot_histogram, plot_state_paulivec
from qiskit.visualization.utils import _bloch_multivector_data
from tqdm.keras import TqdmCallback
from scipy import optimize as opt
import numdifftools as nd

In [ ]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit import IBMQ
from qiskit.aqua.operators import Z2Symmetries

def get_qubit_op(dist):
    driver = PySCFDriver(atom=f'H .0 .0 .0; H .0 .0 {dist}', 
                         unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    num_particles = molecule.num_alpha + molecule.num_beta
    qubitOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals).mapping(map_type='parity')
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
    return qubitOp.to_opflow(), molecule.nuclear_repulsion_energy

tmp = []
ds = np.linspace(0.5, 3, 20)
for dist in ds:
    op, repulsion = get_qubit_op(dist)
    tmp.append(np.linalg.eigvalsh(op.to_matrix()).min() + repulsion)
plt.plot(ds, tmp)

In [ ]:
import itertools

class CRBM(tf.keras.models.Model):
    def __init__(self, nv=1, nh=5, seed=0):
        np.random.seed(seed)
        tf.random.set_seed(seed)
        super(CRBM, self).__init__()
        self.nv, self.nh = nv, nh
        self.bv = tf.Variable(initial_value=CRBM.random_normal((nv,1)), dtype=tf.dtypes.complex64, trainable=True)
        self.bh = tf.Variable(initial_value=CRBM.random_normal((nh,1)), dtype=tf.dtypes.complex64, trainable=True)
        self.W = tf.Variable(initial_value=CRBM.random_normal((nv,nh)), dtype=tf.dtypes.complex64, trainable=True)
        
        v_vecs = np.array([list("{0:b}".format(x).zfill(nv)) for x in np.arange(2**nv)]).astype(np.int)
        self.v_vecs = tf.cast(tf.convert_to_tensor(v_vecs), tf.dtypes.complex64)
    
    @staticmethod
    def random_normal(shape, scale=1, loc=0):
        return np.random.normal(size=shape, scale=scale, loc=loc) + 1j*np.random.normal(size=shape, scale=scale, loc=loc)
    
    def psi(self, v):
        """
        Accepts only length N binary strings
        """
        if tf.rank(v) == 1:
            v = tf.expand_dims(v, axis=-1)
        elif tf.rank(v) == 2:
            v = tf.transpose(v)
        bv, bh, W = self.bv, self.bh, self.W
        ret = tf.linalg.adjoint(bv) @ v + tf.reduce_sum(tf.math.log(tf.math.conj(tf.exp(bh + tf.linalg.adjoint(W) @ v)) + 1), axis=0)
        return tf.transpose(tf.exp(ret))

    def prob(self, measurements):
        psi_vec = self.psi(self.v_vecs)
        psi_vec = psi_vec / tf.linalg.norm(psi_vec)
        psi_proj = tf.math.conj(measurements) @ psi_vec
        return tf.math.abs(psi_proj) ** 2

    def call(self, measurements):
        self.add_loss(-tf.math.reduce_mean(tf.math.log(self.prob(measurements))))

In [ ]:
operator = get_qubit_op(0.7)[0].to_matrix()
N = 2
psi = np.linalg.eigh(operator)[1][:,0]
sv = Statevector(psi)

In [ ]:
print(get_qubit_op(1)[0])

In [ ]:
from tensorflow.keras.callbacks import Callback
class EarlyStoppingByLoss(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

def measure(psi, n_sample=100, basis='Z'*N):
    mat = Pauli.from_label(basis).to_matrix()
    eigs, basis = np.linalg.eigh(mat) # basis[:,i] is ith basis vec
    probs = np.abs(np.conj(basis.T) @ psi)**2
    basis = basis.astype(np.complex64)
    idx = np.random.choice(2**N, size=n_sample, p=probs)
    return basis.T[idx]

def train(psi, n_sample, seed=0):
    np.random.seed(seed)
    measurements = np.concatenate((
        measure(psi, n_sample, basis='II'), 
        measure(psi, n_sample, basis='IZ'), 
        measure(psi, n_sample, basis='ZI'), 
        measure(psi, n_sample, basis='ZZ'), 
        measure(psi, n_sample, basis='XX'), 
    ), axis=0)

    bs = 20
    crbm = CRBM(nv=N, nh=4)
    crbm.compile(optimizer=tf.optimizers.Adam(1e-1))
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='loss', verbose=False, save_best_only=True),
        # TqdmCallback(batch_size=bs),
        EarlyStoppingByLoss(monitor='loss', value=0.3)
    ]
    hist = crbm.fit(measurements, verbose=False, epochs=1000, callbacks=callbacks, batch_size=bs, shuffle=True)
    crbm.load_weights("model.h5")
    crbm.compile(optimizer=tf.optimizers.RMSprop(1e-2))
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='loss', verbose=False, save_best_only=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=100, monitor='loss', factor=0.3),
        # TqdmCallback(batch_size=bs),
        tf.keras.callbacks.EarlyStopping(monitor='loss', patience=200, restore_best_weights=True)
    ]
    hist = crbm.fit(measurements, verbose=False, epochs=4000, callbacks=callbacks, batch_size=len(measurements), shuffle=False)
    crbm.load_weights("model.h5")

    psi2 = crbm.psi(crbm.v_vecs).numpy().flatten()
    psi2 = psi2/psi2[0]
    print(n_sample, seed, min(hist.history['loss']), len(hist.history['loss']))
    return psi2/np.linalg.norm(psi2)

In [ ]:
from tqdm.notebook import trange
res = []
x = np.logspace(2.5, 4.5, num=8).astype(int)
x = (x//20) * 20
for n_sample in x:
    res.append([])
    for i in trange(100):
        sv2 = Statevector(train(psi, n_sample, seed=i*n_sample))
        res[-1].append(sv2.expectation_value(operator))

In [ ]:
ops = get_qubit_op(0.7)[0]
expect = np.real(np.array([sv.expectation_value(op.to_matrix())/op.coeff for op in ops]))
coeffs = np.array([op.coeff for op in ops])
vars = np.sum(coeffs**2 * (1-expect**2))

In [ ]:
plt.figure(figsize=(8,5))
y = np.var(np.real(res), axis=1)
plt.loglog(x,y, 'o', label='RBM')
plt.loglog(x,vars/x, 'o', label='Averaging (QC)')
plt.xlabel("Number of Samples")
plt.ylabel("Variance of observable")
plt.legend()
plt.grid()
plt.grid(which='minor', ls='--', alpha=0.5)
plt.savefig("rbm-results.png")

In [ ]:
plt.figure(figsize=(8,5), dpi=100)
y = np.mean((np.real(res) - np.linalg.eigvalsh(operator)[0])**2, axis=1)
plt.loglog(x,y/0.0016**2, 'o', label='RBM')
plt.loglog(x,(vars/x)/0.0016**2, 'o', label='Naive Averaging')
plt.xlabel("Number of Samples")
plt.ylabel(r"Mean squared error $\langle (\frac{\tilde{O} - O}{\epsilon})^2 \rangle$")
plt.legend()
plt.grid()
plt.grid(which='minor', ls='--', alpha=0.5)
plt.tight_layout()
plt.savefig("rbm-results.png")

In [ ]:
eps = 0.0016
fig, (ax0, ax1) = plt.subplots(2, 1, sharex=True, figsize=(8,8))
ax0.loglog(x, np.mean(np.real(res) - np.linalg.eigvalsh(operator)[0], axis=1)/eps, 'o')
ax0.set_ylabel(r"Bias (normalized by $\epsilon$)")
ax0.grid()
ax0.grid(which='minor', ls='--', alpha=0.5)
ax1.loglog(x, np.std(np.real(res), axis=1)/eps, 'o')
ax1.set_ylabel(r"Standard dev (normalized by $\epsilon$)")
ax1.grid()
ax1.grid(which='minor', ls='--', alpha=0.5)
ax1.set_xlabel("Number of samples")
fig.tight_layout()
fig.savefig("bias.png")